In [ ]:
# Install sentence-transformers
%pip install sentence-transformers

In [ ]:
%pip install pinecone-client

In [ ]:
%pip install transformers -U

In [19]:
%pip install -q langchain

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json

def read_json_file(file_path):
    """Read from a JSON file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

In [11]:
json_data = read_json_file('AI__json_output_2024-12-05_12-59-34.json')

sections = []

heading_title = ''

for section in json_data['sections']:
    heading_title = section['heading']
    paragraphs = " ".join(section['paragraphs'])
    sections.append(f'{heading_title}\n{paragraphs}')


print(sections)

['Executive Summary\nPart 1: Foundational Information\n1 Framing Risk Artificial intelligence (AI) technologies have significant potential to transform society and\npeople’s lives – from commerce and health to transportation and cybersecurity to the envi-\nronment and our planet. AI technologies can drive inclusive economic growth and support\nscientific advancements that improve the conditions of our world. AI technologies, how-\never, also pose risks that can negatively impact individuals, groups, organizations, commu-\nnities, society, the environment, and the planet. Like risks for other types of technology, AI\nrisks can emerge in a variety of ways and can be characterized as long- or short-term, high-\nor low-probability, systemic or localized, and high- or low-impact. The AI RMF refers to an AI system as an engineered or machine-based system that\ncan, for a given set of objectives, generate outputs such as predictions, recommenda-\ntions, or decisions influencing real or virtua

In [8]:
#Split Text using Regular Expressions


import re


#Open text file

# Open a file using a context manager
with open('AI_RMP_extract4.txt', 'r') as file:
    content = file.read()
    #print(content)

# Sample text simulating the content of a text file

text = """
Page 1
This is the content of page 1.
It has multiple lines.

Page 2
This is the content of page 2.
It also has multiple lines.

Page 3
This is the content of page 3.
"""

# Regular expression to match 'Page #' where # is one or more digits
#pattern = r'Page \d+'
pattern = r'Page (\d+|[ivxlcdm]+)'

# Split the text using the pattern
pages = re.split(pattern, content)

# Remove any empty strings from the list
sections = [page.strip() for page in pages if page.strip()]

print(len(sections))

# Print the pages

#for i, page in enumerate(pages, start=1):
#    print(f"Page {i}:\n{page}\n")

87


In [12]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model for generating embeddings
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings for each section
section_embeddings = model.encode(sections)

/usr/local/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/usr/local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This 

In [13]:
import pinecone
from pinecone import Pinecone, ServerlessSpec
import os
os.environ['PINECONE_API_KEY'] = '13707c0e-f5d1-4eaa-8c72-4988914c1de3'

pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)
index_name = 'pdf-sections-12062024'
# Now do stuff
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name, 
        dimension=384, 
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
# Initialize Pinecone
#pinecone.init(api_key="13707c0e-f5d1-4eaa-8c72-4988914c1de3", environment="us-east-1")

# Create a new index
#index_name = 'pdf_sections'
#pinecone.create_index(index_name, dimension=768)

# Connect to the index
index = pc.Index(index_name)

# Upload section embeddings
for i, embedding in enumerate(section_embeddings):
    index.upsert([(str(i), embedding)])

print("Sections uploaded to Pinecone!")


Sections uploaded to Pinecone!


In [29]:
# Search for a query within the document sections
query = "Trustworthiness"
query_embedding = model.encode([query])
#print(query_embedding[0])
query_embedding_list = query_embedding[0].tolist()

# Search the vector database
#search_results = index.query(vector=query_embedding[0], top_k=5)
search_results = index.query(
    
    vector=query_embedding_list,
    top_k=5,
    include_values=True
)
#print(search_results)
# Retrieve and display the most relevant sections
for match in search_results['matches']:
    section_id = int(match['id'])
    #print(f"Section {section_id}: {sections[section_id]}")
    print(section_id)


10
11
17
4
14


In [30]:
def get_search_results(query):
    # Search for a query within the document sections
    query = "Trustworthiness"
    query_embedding = model.encode([query])
    #print(query_embedding[0])
    query_embedding_list = query_embedding[0].tolist()
    
    # Search the vector database
    #search_results = index.query(vector=query_embedding[0], top_k=5)
    search_results = index.query(
        
        vector=query_embedding_list,
        top_k=5,
        include_values=True
    )

    return search_results

In [17]:
import json

# Example structure for storing sections and subsections
document_json = {
    "title": "Extracted Document",
    "sections": []
}

# Append sections to the JSON object
for match in search_results['matches']:
    section_id = int(match['id'])
    document_json["sections"].append({
        "section_id": section_id,
        "content": sections[section_id]
    })

# Save the structured data to a JSON file
with open('PDF_document2.json', 'w') as json_file:
    json.dump(document_json, json_file, indent=2)

print("Document saved to JSON!")


Document saved to JSON!


In [31]:
def output_to_file(keyword, search_results):
    # Example structure for storing sections and subsections
    document_json = {
        "title": f"Extracted document for keyword {keyword}",
        "sections": []
    }

    if(search_results):
        # Append sections to the JSON object
        for match in search_results['matches']:
            section_id = int(match['id'])
            document_json["sections"].append({
                "section_id": section_id,
                "content": sections[section_id]
            })
    
        # Remove spaces and set all characters to lower case 
        modified_string = keyword.replace(" ", "").lower()
        file_name = f'query_results_{modified_string}.json'
        # Save the structured data to a JSON file
        with open(file_name, 'w') as json_file:
            json.dump(document_json, json_file, indent=2)
        
        print(f"{file_name} saved to JSON!")
    else:
        print("No search results!")

In [21]:
from neo4j import GraphDatabase

#Create db driver ...

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://1eeb71d4.databases.neo4j.io"
AUTH = ("neo4j", "ry8qCNwcZajMLLzq_fmzZER6DYcn3JMKfSTY0-HpZlw")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [32]:

# Query the graph
query = """
 MATCH (k:Keyword)
 RETURN k.name AS Keyword
"""

records, summary, keys = driver.execute_query(
    query,
    keyword='Trustworthiness',
    database_="neo4j",
)
driver.close()
# Loop through results and do something with them
for record in records:
    keyword = record['Keyword']
    results = get_search_results(keyword)
    output_to_file(keyword, results)
    #print(record['Keyword'])





/var/folders/gf/gs7hmkc56sl9k2z0v7pgmlw00000gn/T/ipykernel_6234/3765920612.py:7: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


query_results_aiethics.json saved to JSON!
query_results_transparency.json saved to JSON!
query_results_accountability.json saved to JSON!
query_results_biasmitigation.json saved to JSON!
query_results_fairness.json saved to JSON!
query_results_dataprivacy.json saved to JSON!
query_results_dataprotection.json saved to JSON!
query_results_regulatorycompliance.json saved to JSON!
query_results_gdpr(generaldataprotectionregulation).json saved to JSON!
query_results_audittrails.json saved to JSON!
query_results_explainability.json saved to JSON!
query_results_robustness.json saved to JSON!
query_results_security.json saved to JSON!
query_results_humanoversight.json saved to JSON!
query_results_datagovernance.json saved to JSON!
query_results_riskmanagement.json saved to JSON!
query_results_ethicalai.json saved to JSON!
query_results_algorithmicaccountability.json saved to JSON!
query_results_modelmonitoring.json saved to JSON!
query_results_complianceframework.json saved to JSON!
query_res